# Práctica de Selección de Instancias

## Índice de la práctica

- [Leave-one-out para kNN](#Leave-one-out-para-kNN)
- [Visualización de las fronteras de decisión](#Visualización-de-las-fronteras-de-decisión)
- [Métodos de selección de instancias](#Métodos-de-selección-de-instancias): ENN, AllKNN, CNN, RNN, RMHC
- [Comparación de los métodos con kNN](#Comparación-de-los-métodos-con-kNN)
- [Comparación de los métodos con Árboles de Decisión](#Comparación-de-los-métodos-con-Árboles-de-Decisión)
- [Selección de Instancias con KEEL: Comparativa](#Selección-de-Instancias-con-KEEL:-Comparativa)


Comenzamos la práctica importando los paquetes necesarios. Como siempre, usaremos NumPy, Matplotlib y Scikit-learn. Además, para la parte de KEEL haremos uso del paquete subprocess para ejecutar comandos en el sistema.

In [ ]:
import numpy as np
from sklearn import metrics, neighbors, tree, preprocessing
from scipy import stats

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import subprocess
import dataset as ds
from shutil import copyfile

from test_helper import Test

%matplotlib inline  

# Leave-one-out para kNN
Lo primero que vamos a realizar en la práctica es la implementación de un modelo de leave-one-out para los k vecinos más cercanos. Este método es muy útil para valorar la calidad de los conjuntos de instancias seleccionadas. La idea es clasificar cada instancia con el resto de instancias en conjunto de entrenamiento (sin considerar a ella misma).

Si solo queremos hacer leave-one-out con el conjunto de entrenamiento, podríamos usar la librería de sklearn. Sin embargo, nosotros queremos evaluar un conjunto de instancias seleccionadas $S$ sobre el conjunto completo de entrenamiento $TR$. Aplicar leave-one-out en este caso, significa que queremos conocer la clasificación de todas las instancias en $TR$ usando $S$ como referencia ($S\subset TR$). Por tanto, cuando vayamos a clasificar una instancia que está seleccionada en $S$ no debemos considerar a ella misma como su propio vecino. Esto es lo que sklearn no permite hacer.

En la siguiente función vamos a implementar esa lógica utilizando algunas de las funciones de la clase neighbors.KNeighborsClassifier de sklearn: [Link a neighbors.KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier)

En dicha clase, tenemos una función llamada `kneighbors(X)` que nos devuelve los índices de los vecinos más cercanos cercanos a cada instancia en `X` y las distancias asociadas. Lo que vamos a hacer es obtener para cada ejemplos sus $k+1$ ejemplos más cercanos con sus distancias. Posteriormente, si la distancia del vecino más cercano es $0$, asumiremos que dicho vecino es el propio ejemplo y por tanto sustituiremos dicho vecino por el vecino $k+1$ que en otro caso no será considerado para la votación.

Además, como la función `kneighbors` nos devuelve el índice del vecino, pero no su clase, crearemos una función que nos permita transformar los índices a la clase correspondiente del propio vecino, para posteriormente poder calcular la clase más repetida entre los $k$ vecinos más cercanos (una vez eliminados los que tienen distancia 0).

In [ ]:
def leaveOneOut(knn, X, y):
    """
    :param knn: Instancia de KNeighborsClassifier entrenada (con fit hecho con los datos de train o el subconjunto seleccionado)
    :param X: Matriz con los ejemplos de entrenamiento completo (para hacer leave one out)
    :param y: Vector con la salida de los ejemplos de entrenamiento completo (correspondientes a X)
    :return: Vector con la salida obtenida para cada ejemplo de X (siguiendo el esquema leave-one-out)
    """
    # Para hacer la práctica más flexible añadimos la posibilidad de llamar a esta función con un árbol de decisión
    # En ese caso simplemente se utiliza el árbol aprendido y se predicen los ejemplos con dicho árbol
    if type(knn) == tree.DecisionTreeClassifier:
        # Utiliza el método predict de knn (que en este caso será un árbol de decisión) para obtener la predicción
        prediction = #<RELLENAR>
        return prediction
    
    # kneighbors nos devuelve los vecinos más cercanos de cada ejemplo en X y las distancias asociadas
    # Para ello recibe tres parámetros, X, el número de vecinos (n_neighbors y si queremos las distancias asociadas o no)
    distancias, vecinos = knn.kneighbors(X, n_neighbors=knn.n_neighbors + 1, return_distance=True)
    # Realmente no estamos interesados en el propio vecino, sino en la clase a la que pertenece
    # Transformamos los vecinos a una matriz con la clase de cada vecino
    # Creamos una función que trabaja sobre numpy, recibe un idVecino y devuelve su clase, obtenida de knn._y (la clase de los ejemplos en kNN)
    pasarAclase = np.vectorize(lambda idVecino: knn._y[idVecino])
    # Utilizamos la función anterior para obtener la clase de cada vecino
    vecinosClase = pasarAclase(vecinos)
    # Ahora queda decidir la clase de salida para cada uno de los ejemplos
    # Debemos tener en cuenta: Si la distancia al primer vecino es 0, el vecino es el propio ejemplo y por tanto no lo usamos para decidir
    #   1. Obtenemos una máscara con los ejemplos en los que ocurre esto
    #   2. Sustituimos la clase del primer vecino por la del último (el k+1 que sino no se utiliza)
    #   3. Obtenemos la moda (el valor más repetido de clase) para cada fila (cogiendo los k primeros valores)
    mascara = (distancias[:, 0] == 0)
    vecinosClase[mascara, 0] = vecinosClase[mascara, -1]
    prediction, conteo = stats.mode(vecinosClase[:, :-1], axis=1) # el k+1 no se utiliza

    return prediction

A continuación vamos a comprobar si la función implementada funciona correctamente. Para ello en primero lugar vamos a hacer la lectura de los datos.

En esta práctica vamos a trabajar con dos conjuntos de datos muy diferentes: banana e iris.

Banana es un dataset artificial con dos clases y dos atributos reales. Tiene alrededor de 5000 instancias por lo que las ejecuciones serán algo más lentas con él. El dataset resulta interesante porque nos permita estudiar de manera simple y visual el comportamiento de los modelos de selección de instancias.

Iris es un dataset que ya habéis utilizado anteriormente, se trata de un problema de clasificación en el que se deben distinguir tres tipos de plantas iris según las cuatro características reales establecidas. Para poder trabajar visualmente con dicho dataset, en esta práctica utilizaremos una modificación del mismo en el que utilizaremos únicamente las dos últimas características. El dataset ya está transformado para tal fin.

Realizamos la lectura de una partición de entrenamiento y una de test para cada dataset y normalizamos los datos con el método de los máximos y mínimos (llevando los datos a [0, 1]). La normalización es necesaria para aplicar $k$NN. [Link a normalización MinMaxScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler)

Vamos a guardar dos diccionarios, uno con los datasets `nombre dataset -> (conjunto train, conjunto test)` y otro con la normalización `nombre dataset -> min_max_scaler` (este segundo lo reutilizaremos al final de la práctica). Necesitamos el diccionario ya que cada dataset tendrá su propia normalización asociada a los valores máximos y mínimos de sus atributos

In [ ]:
# Establecemos los nombres de los datasets a utilizar
nombresDatasets = {'banana', 'iris'}
# Inicializamos los diccionarios
datasets = {}
min_max_scaler = {}

# Para cada dataset realizamos las operaciones
for dataset in nombresDatasets:
    # Establecemos la ruta del train y el test
    fileTr = 'datasets/' + dataset + '/' + dataset + '-5dobscv-2tra.dat'
    fileTst = 'datasets/' + dataset + '/' + dataset + '-5dobscv-2tst.dat'
    # Utiliza ds.lectura para leer los datos de ambos conjuntos
    train, originalTr = #<RELLENAR>
    test, originaTst = #<RELLENAR>
    # Creamos la instancia para la normalización de MinMaxScaler()
    min_max_scaler[dataset] = #<RELLENAR>
    # Ajustamos la normalización con train.data (fit_transform) y la aplicamos a test.data (transform)
    train.data = min_max_scaler[dataset].#<RELLENAR>
    test.data = min_max_scaler[dataset].#<RELLENAR>
    # Finalmente almacenamos el dataset en formato de tupla (train, test)
    datasets[dataset] = (train, test)
    
    
    print("Ejemplos por clase en " + dataset + ":")
    print(np.bincount(train.target))

Test.assertEquals(list(np.bincount(datasets['iris'][0].target)), [40, 40, 40], "Iris no está leído correctamente")
Test.assertEquals(list(np.bincount(datasets['banana'][0].target)), [2339, 1901], "Banana no está leído correctamente")

Una vez leídos los conjuntos de datos es hora de probar el método de leaveOneOut implementado. Para ello, vamos a crear el clasificador kNN con `n_neighbors=1`. Posteriormente, evaluaremos cada dataset en train (con `leaveOneOut`) y en test (directamente con `knn.predict`). En ambos casos, para obtener la precisión podemos utilizar las métricas que nos provee sklearn. En este caso, nos centraremos en la precisión o porcentaje de acierto que podemos obtener utilizando `metrics.accuracy_score` a la cual debemos pasar un vector con la salida dada por el clasificador para cada ejemplo y otro vector con la salida real para cada uno de ellos.

In [ ]:
# Creamos el clasificador knn con neighbors.KNeighborsClassifier y n_neighbors=1
knn = neighbors.KNeighborsClassifier(n_neighbors=1)
# Evaluamos cada dataset
for dataset in nombresDatasets:
    # Obtenemos los conjuntos de train y test del dataset
    (train, test) = datasets[dataset]
    # "Entrenamos" knn con el conjunto de train con fit (usando train.data y train.target)
    knn = #<RELLENAR>
    # Obtenemos las salidas para las instancias de train utilizando leaveOneOut con k por defecto (=1)
    salidasTrain = #<RELLENAR>
    # Obtenemos la precisión en train mediante metrics.accuracy_score comparando la salida obtenida con el target de train
    accTrain = #<RELLENAR>
    # Obtenemos las salidas para test mediante knn.predict
    predictionTest = #<RELLENAR>
    # Obtenemos la precisión en test mediante metrics.accuracy_score comparando la salida obtenida con el target de test
    accTest = #<RELLENAR>
    
    # Imprimimos los resultados obtenidos
    print("Dataset " + dataset)
    print("Resultados sin pre-procesar con 1NN")
    print("Precisión en train: {}".format(accTrain))
    print("Precisión en test: {}".format(accTest))
    
    # A continuación mostramos las fronteras de decisión que obtenemos con los conjuntos de train y cómo quedan los
    # ejemplos el conjunto de test en base a dichas fronteras
    # DESCOMENTAR TRAS IMPLEMENTAR LA FUNCIÓN MOSTRAR QUE VIENE A CONTINUACIÓN
    # mostrar(knn, train.data, train.target, test.data, test.target)
    
Test.assertEquals(round(accTrain, 4), 0.8587, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.8972, "Precisión en test incorrecta para banana")

# Visualización de las fronteras de decisión
Para poder comprender mejor el funcionamiento de los métodos de selección de instancias vamos a basarnos en la visualización de las instancias (es por esto que estamos usando datasets con solo dos características).

Vamos a crear un función mostrar que nos va a permitir mostrar gráficamente los resultados obtenidos. Queremos:
1. **Mostrar los ejemplos utilizados como referencia en kNN en una gráfica junto con la frontera de decisión** que definen. En el caso de no haber procesado el dataset, este será el conjunto de train, pero en caso de haber realizado una selección de instancias mostraremos únicamente dichas instancias ya que son las utilizadas para entrenar el clasificador.
2. En caso de no **mostrar los ejemplos de entrenamiento** en la primera figura debido a que hemos realizado la selección de instancias, mostraremos dichos ejemplos en otra gráfica con la frontera de decisión definida por los ejemplos seleccionados.
3. **Mostramos los ejemplos de test sobre las fronteras de decisión del clasificador** (observaremos cuáles se fallan a la hora de generalizar a un conjunto de datos no visto en el entrenamiento ni en la selección de instancias).

In [ ]:
def mostrar(clasificador, X, y, Xtst, ytst, method=None, Xsel=None, ysel=None):
    """
    Esta función muestra las fronteras de decisión del clasificador ya entrenado y los ejemplos en X
    (con el color dependiendo de y). En caso de pasar valores para los parámetros Xsel e ysel, dibuja una nueva
     figura con los prototipos utilizados para crear el clasificador
    :param clasificador: Clasificador entrenado (kNN o decision tree)
    :param X: Matriz con los ejemplos a mostrar
    :param y: Vector con las salidas de los ejemplos a mostrar
    :param method: String con el nombre del método de selección de instancias utilizado
    :param Xsel: Matriz con los prototipos seleccionados
    :param ysel: Vector con las clases de los prototipos seleccionados
    :return: Nada
    """
    h = .02  # Tamaño de paso para la malla

    # Creamos los mapas de colores a utilizar
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])

    # Preparamos los ejemplos de entrada para poder pintar la frontera de decisión
    # Asignamos una clase (color) a cada ejemplo de la malla en [x_min, x_max]x[y_min, y_max].
    x_min, x_max = X[:, 0].min() * 0.9, X[:, 0].max() * 1.1
    y_min, y_max = X[:, 1].min() * 0.9, X[:, 1].max() * 1.1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    # Clasificamos los puntos
    Z = clasificador.predict(np.c_[xx.ravel(), yy.ravel()])
    # Ponemos el resultado en el formato deseado
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(17,5.5))

    # En caso de pasar los prototipos seleccionados, pintamos su figura a la izquierda
    if Xsel is not None:
        plt.subplot(131)
        # Pintamos las fronteras
        plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
        # Pintamos los puntos
        plt.scatter(Xsel[:, 0], Xsel[:, 1], c=ysel, cmap=cmap_bold, facecolor='none')
        # Asignamos el título
        plt.title("Ejemplos seleccionados con %s" % (method))
        # Establecemos los límites
        plt.xlim(xx.min(), xx.max())
        plt.ylim(yy.min(), yy.max())
        plt.subplot(132)
    else:
        plt.subplot(121)

    # Pintamos la figura de los ejemplos en X (a la izquierda si ya hemos pintado la primera)
    # Pintamos las fronteras
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
    # Pintamos los puntos
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold)
    # Asignamos el título
    plt.title("Ejemplos de Train")
    # Establecemos los límites
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

    # Pintamos la figura de los ejemplos en Xtst (a la izquierda si ya hemos pintado la primera)
    if Xsel is not None:
        plt.subplot(133)
    else:
        plt.subplot(122)

    # Pintamos las fronteras
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
    # Pintamos los puntos de test
    plt.scatter(Xtst[:, 0], Xtst[:, 1], c=ytst, cmap=cmap_bold)
    # Asignamos el título
    plt.title("Ejemplos de Test")
    # Establecemos los límites
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())

    if type(clasificador) == neighbors.KNeighborsClassifier:
        plt.suptitle('Clasificador: %iNN' % (clasificador.n_neighbors), fontsize=16)
    else:
        plt.suptitle('Clasificador: Decision Tree', fontsize=16)
    # Mostramos la figura
    plt.show()
    return

Una vez implementada la función, vuelve a la celda anterior, descomenta la línea de `mostrar` y ejecútala de nuevo. Observa los resultados obtenidos.

# Métodos de selección de instancias
A continuación vamos a implementar cinco métodos de selección de instancias en Python y estudiaremos su comportamiento.

Como notación, recordemos que $TR$ hace referencia al conjunto completo de instancias de entrenamiento y $S$ al conjunto de instancias seleccionadas por el método de selección.

## Edited Nearest Neighbors (ENN)

Este método funciona de la siguiente forma:
1. $S=TR$
2. Para cada ejemplo $e$ en $S$:
    * Eliminamos el ejemplo $e$ si es incorrectamente clasificado por los ejemplos en $S-\{e\}$ (las seleccionadas sin contar a ella misma)

Para la implementación, en la mayoría de los métodos representaremos las instancias seleccionadas como un vector de booleanos $S$ del tamaño del número de ejemplos, de tal forma que el valor $S[i]$ en la posición $i$ indica si el ejemplo $i$ del conjunto de entrenamiento ha sido seleccionado (True) o no (False).

Para ENN, $S$ comienza siendo un vector con todos sus elementos a True. En cada iteración, pondremos el elemento correspondiente a False y comprobaremos si es correctamente clasificado con los elementos restantes en $S$ (como $S$ es una máscara de booleanos, podemos obtener el dataset con los ejemplos seleccionados como X[S, :] e y[S]). En caso de ser correctamente clasificado, restauramos su valor a True y en otro caso, queda eliminado. 

In [ ]:
def ENN(X, y, k=3):
    """
    Algoritmo ENN para la selección de instancias. Cada ejemplo se elimina si se clasifica incorrectamente por sus k vecinos más cercanos
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    S = np.ones( (X.shape[0]), dtype=bool)  # inicialmente todos los índices/instancias son seleccionadas

    # Creamos el clasificador knn con el valor de k dado
    knn = #<RELLENAR>

    # Para cada ejemplo, comprobamos si este es fallado al ser clasificado por sus k vecinos más cercanos (sin contar a sí mismo)
    for i in range(0, X.shape[0]):
        # Eliminamos el ejemplo de la selección para no tenerlo en cuenta al clasificarlo
        S[i] = #<RELLENAR>
        # Utilizamos para knn el conjunto de seleccionadas actualmente
        knn = knn.fit(X[S, :], y[S])
        # Si la instancia es correctamente clasificada, restauramos su valor en S[i] = True, en otro caso será eliminada
        # Obtener la predicción de la instancia i (es decir X[i, :]) con knn.predict
        if y[i] == #<RELLENAR>:
            S[i] = #<RELLENAR>
    return S

Para poder probar el método implementado vamos a implementar una función `ejecutaMetodoIS` que la reutilizaremos con el resto de métodos. Esta función se encargará de recibir un método de selección de instancias (una función), un clasificador, un conjunto de entrenamiento y uno de test y:
1. llevará a cabo la selección de instancias con el método especificado, 
2. entrenará el clasificador con las instancias seleccionadas,
3. obtendrá la precisión en train del clasificador (mediante la función leaveOneOut),
4. obtendrá la precisión en test mediante predict
5. y calculará el ratio de reducción obtenido (qué porcentaje de instancias del conjunto de entrenamiento han sido seleccionadas).

Finalmente, imprimirá dichos datos (en caso de que el parámetro verbose sea True) y mostará gráficamente los resultados obtenidos utilizando la función mostrar.

En caso de estar trabajando con un árbol de decisión, calculará también el número de reglas utilizadas (de esta forma, más adelante, trataremos de estudiar cómo afectan estos métodos de selección a los árboles de decisión.

In [ ]:
def ejecutaMetodoIS(funcionMetodo, clasificador, train, test, verbose=True):
    """
    Esta función se encajar de ejecutar un método de selección de instancias y dar los resultados sobre train y test
    con el clasificador introducido como parámetro
    :param funcionMetodo: Función con el método de selección de instancias a ejecutar
    :param clasificador: Instancia del clasificador a utilizar para obtener la precisión en train y test
    :param train: Conjunto de datos de entrenamiento leído del formato keel con campos data y target
    :param test: Conjunto de datos de test leído del formato keel con campos data y target
    :return: tupla con la máscara de ejemplos seleccionados, la precisión en train, precisión en test y porcentaje de reducción obtenido
    """
    # Obtenemos el nombre del método. Como es una función utilizamos su campo __name__
    nombreMetodo = funcionMetodo.__name__
    if verbose:
        print("Ejecutando " + nombreMetodo + "...")
    # Ejecutamos el método correspondiente con los ejemplos de entrenamiento y sus clases
    # Estamos lanzando todos los métodos con los parámetros por defecto
    S = funcionMetodo(train.data, train.target)
    # Entrenamos el clasificador correspondiente solo con los ejemplos seleccionados (aplicamos la máscara en S)
    clasificador = clasificador.fit(train.data[S, :], train.target[S])
    # Obtenemos las salidas para train mediante leaveOneOut (aplicado solo para kNN, sino el árbol se aprende en train directamente)
    predictionTrain = #<RELLENAR>
    # Obtenemos la precisión en train con las salidas obtenidas con metrics.accuracy_score
    accTrain = #<RELLENAR>
    # Obtenemos las salidas para test utilizando la función predict del clasificador
    predictionTest = #<RELLENAR>
    # Obtenemos la precisión en test utilizando las métricas de sklearn metrics.accuracy_score
    accTest = #<RELLENAR>
    # Calculamos el porcentaje de reducción
    reduction = 100.0 - S.sum() / float(S.size)*100.0
        
    
    if verbose:
        print("Dataset " + dataset)
        print("Resultados " + nombreMetodo + " python")
        print("Precisión en train: {}".format(accTrain))
        print("Precisión en test: {}".format(accTest))
        print("Reducción " + nombreMetodo + ": {} de {}".format(S.sum(), S.size))
        print("Reducción: %2.2f%%" % reduction)
        # En caso de ser un árbol, imprimimos el número de reglas obtenidas
        if type(clasificador) == tree.DecisionTreeClassifier:
            print("Número de reglas: {}".format(clasificador.tree_.node_count))

    # Mostramos los resultados obtenidos por el método gráficamente
    mostrar(clasificador, train.data, train.target, test.data, test.target,  nombreMetodo, train.data[S, :], train.target[S])

    # Si es un árbol, devolvemos el número de reglas obtenidas
    nReglas = -1
    if type(clasificador) == tree.DecisionTreeClassifier:
        nReglas = clasificador.tree_.node_count
    return S, accTrain, accTest, reduction, nReglas

Con la función `ejecutaMetodoIS` implementada, podemos comprobar los resultados obtenidos por ENN en cada dataset.

¿Elimina muchos ejemplos? ¿Mejora la precisión en train? ¿y en test? ¿Qué ha pasado con las fronteras de decisión?

In [ ]:
# Para cada dataset...
for dataset in nombresDatasets:
    # Obtenemos el conjunto de train y el de test del diccionario datasets utilizando el nombre del dataset
    train, test = #<RELLENAR>
    # Ejecutamos el método ENN con el clasificador knn creado anteriormente y los conjuntos obtenidos (train y test)
    # Con _ evitamos recibir los parámetros en variables, para el Test.assertEquals solo comprobaremos la precisión por ahora
    _, accTrain, accTest, _, _ = ejecutaMetodoIS(#<RELLENAR>)
    
Test.assertEquals(round(accTrain, 4), 0.9142, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.8962, "Precisión en test incorrecta para banana")

## AllKNN

Este método se basa en el anterior para decidir si eliminar una instancia o no. Básicamente, elimina aquellas instancias que son mal clasificadas utilizando kNN con cualquier valor de k entre 1 y el valor dado como parámetro.

1. $S=TR$
2. Para cada ejemplo $e$ en $S$:
    * Lo clasificamos con $k = 1...K$ utilizando $TR - \{e\}$
    * Si se clasifica incorrectamente con alguno de los valores, se etiqueta para ser eliminado (pero se mantiene en el conjunto para decidir la eliminación del resto de instancias, es decir, el conjunto de referencia es siempre $TR$).

De nuevo, representaremos las instancias seleccionadas como un vector de booleanos $S$ del tamaño del número de ejemplos, de tal forma que el valor $S[i]$ en la posición $i$ indica si el ejemplo $i$ del conjunto de entrenamiento ha sido seleccionado (True) o no (False).

Para AllKNN, $S$ comienza siendo un vector con todos sus elementos a True. En cada iteración, comprobamos si se clasifica incorrectamente con alguno de los valores de $k$ (para ello utilizamos una máscara auxiliar que nos permite eliminar el ejemplo y que su vecino más cercano no sea el mismo). En caso de ser mal clasificado, establecemos el valor $S[i]$ correspondiente a False y continúamos con el siguiente ejemplo (pero hay que tener en cuenta que $S$ no se aplica para crear el conjunto de train a la hora de predecir la clase de cada ejemplo con cualquier valor de $k$.

In [ ]:
def AllKNN(X, y, k=5):
    """
    Algoritmo AllKNN para selección de instancias. Para cada instancia se prueba si se falla con 1 hasta k vecinos. 
    Si con alguno de dichos valores se clasifica incorrectamente, se marca para su eliminación (las que van a ser 
    eliminadas se consideran para el resto de instancias)
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Máximo valor de k a probar en AllKNN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    S = #<RELLENAR> # inicialmente todos los índices/instancias son seleccionadas

    # Para cada ejemplo en X, comprobamos si este es fallado con algún valor entre 1 y k de vecinos
    for i in range(0, X.shape[0]):
        # Creamos una máscara auxiliar que nos permitirá usar todos los ejemplos menos el actual
        # Esta máscara inicialmente será un vector con todo a True de tamaño igual al número de ejemplos
        Saux = #<RELLENAR>
        # El elemento i-ésimo lo ponemos a False para no utilizar dicho ejemplo
        Saux[i] = False
        # Probamos con todos los valores de k entre 1 y k
        for k_enn in range(1, k): 
            # Creamos el clasificador knn con k = k_enn y lo "entrenamos" con todo el conjunto de instancias menos la actual
            knn = #<RELLENAR>
            knn = knn.fit(#<RELLENAR>) # Utilizar X e y con la máscara Saux!
            # En caso de fallar la predicción eliminamos la instancia y dejamos de comprobar el resto de valores de k
            if y[i] != #<RELLENAR>: # Predecimos la clase de la instancia i (X[i, :]) con predict
                S[i] = False
                break
    return S

Utiliza la función `ejecutaMetodoIS` implementada, para comprobar los resultados obtenidos por AllKNN en cada dataset.

¿Elimina muchos ejemplos? ¿Mejora la precisión en train? ¿y en test? ¿Qué ha pasado con las fronteras de decisión?

In [ ]:
# Para cada dataset...
for dataset in nombresDatasets:
    # Obtenemos el conjunto de train y el de test del diccionario datasets utilizando el nombre del dataset
    train, test = #<RELLENAR>
    # Ejecutamos el método AllKNN con el clasificador knn creado anteriormente y los conjuntos obtenidos (train y test)
    # Con _ evitamos recibir los parámetros en variables, para el Test.assertEquals solo comprobaremos la precisión por ahora
    _, accTrain, accTest, _, _ = ejecutaMetodoIS(#<RELLENAR>) # Utilizar valor de k por defecto
    
Test.assertEquals(round(accTrain, 4), 0.9142, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.9028, "Precisión en test incorrecta para banana")

## Condensed Nearest Neighbor (CNN)

CNN es un método de condensación y por tanto trata de seleccionar los ejemplos en las fronteras. Básicamente, su idea es seleccionar todas aquellas instancias cuya clasificación es errónea, de tal forma que sea capaz de clasificar correctamente todo el conjunto de train (considerando como clasificación correcto cuando un ejemplo se clasifica consigo mismo).

1. $S= \emptyset$
1. Añadir un ejemplo aleatorio de cada clase a $S$
2. Para cada ejemplo $e$ en $TR$:
    * Lo clasificamos con las instancias en $S$
    * Si el ejemplo se falla, lo añadimos $S$ (aseguramos que se clasifica correctamente)
3. Repetimos el proceso hasta que todos los ejemplos restantes en $TR - S$ sean acertados (por definición, los ejemplos en $S$ se aciertan).

Las instancias seleccionadas se representan como un vector de booleanos $S$ del tamaño del número de ejemplos ($S[i]= True$ el ejemplo $i$ es seleccionado y $S[i]= False$, el ejemplo no se selecciona).

Para CNN, $S$ comienza siendo un vector con todos sus elementos a False. Lo primero que debemos hacer es añadir un ejemplo aleatorio de clada clase.

Posteriormente, como solo nos interesa evaluar los ejemplos que no están en $S$, los obtenemos mediante la negación del vector $S$. El bucle de CNN terminará cuando en un nuevo recorrido de dichos ejemplos, ninguno sea fallado. Con cada ejemplo, simplemente debemos clasificarlo con los ejemplos en $S$ y en caso de fallo añadirlo a $S$ (hacer que $S[i] = True$).

In [ ]:
def CNN(X, y, k=1):
    """
    Algoritmo CNN para la selección de instancias. Se comienza con dos ejemplos aleatorios (uno de cada clase) y cada ejemplo
    que se falla al ser clasificado por los ya seleccionados se añade a la selección (se para cuando ya no se añaden más ejemplos)
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Creamos el clasificador knn con el valor de k dado
    knn = #<RELLENAR>

    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    # Inicialmente no seleccionamos instancias, todo el vector debe estar a False
    S = #<RELLENAR> 

    # Inicialmente, seleccionamos aleatoriamente una instancia de cada clase
    # Obtenemos el número de clases diferenes
    nClases = np.unique(y)
    # Para cada clase, buscamos los índices de los ejemplos de dicha clase y seleccionamos uno de ellos aleatoriamente
    # Dicho ejemplo coge el valor True en S
    for c in nClases:
        # Cogemos los índices de los ejemplos de la clase c
        indicesClase = np.where(y == c)[0]  # np.where devuelve una tupla, nos interesa la primera parte que contiene los índices de las posiciones que no son 0 o False
        # Seleccionamos un índice aleatoriamente
        instanciaAleatoria = indicesClase[np.random.randint(0, indicesClase.size)]
        # Ponemos a True la posición correspondiente a dicha instancia en S
        S[#<RELLENAR>] = #<RELLENAR>

    # Solo evaluamos los ejemplos que no están en S (los que están se aciertan por definición al considerlos
    # como vecinos de sí mismos). Estos ejemplos son los únicos que podemos añadir a S
    # Para obtener notS obtenemos el vector que contiene la negación de todos los elemento de S (utiliza np.logical_not())
    notS = #<RELLENAR>

    # "Entrenamos" knn con los ejemplos seleccionados (aplicando la máscara S)
    knn = #<RELLENAR>
    # Inicializamos el número de fallados para entrar en el bucle. El algoritmo termina cuando no se fallan ejemplos
    fallados = -1
    while fallados != 0:
        # Ponemos el contador de fallados a 0 y comprobamos todos los ejemplos no seleccionados
        fallados = 0
        # Obtenemos los índices de los ejemplos no seleccionados
        indices = np.where(notS)[0] # np.where devuelve una tupla, nos interesa la primera parte que contiene 
                                    # los índices de las posiciones que no son 0 o False
        # Para cada ejemplo no seleccionado (y en orden aleatorio) comprobamos si dicho ejemplo se falla 
        # con las instancias en S actualmente. Si se falla, el ejemplo se añade a S
        for i in np.random.permutation(indices):
            if y[i] != #<RELLENAR>: # Predecimos la salida para la instancia i (X[i, :])
                S[i] = True
                # Re-entrenamos knn con las nuevas instancias seleccionadas (aplicando la máscara S)
                knn = #<RELLENAR>
                fallados += 1
        # Recalculamos los ejemplos a estudiar en la próxima iteración, aquellos no seleccionados, utiliza de nuevo np.logical_not()
        notS = #<RELLENAR>
        print("CNN, fin de iteración, fallados: {}, ejemplos en S: {}".format(fallados, np.sum(S)))

    return S

Utiliza la función `ejecutaMetodoIS` implementada, para comprobar los resultados obtenidos por CNN en cada dataset.

¿Elimina muchos ejemplos? ¿Mejora la precisión en train? ¿y en test? ¿Qué ha pasado con las fronteras de decisión? ¿Hay cambios respecto a los métodos de edición (ENN, AllKNN)?

In [ ]:
# Como usamos aleatorios en CNN establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
np.random.seed(12312)
# Para cada dataset...
for dataset in nombresDatasets:
    # Obtenemos el conjunto de train y el de test del diccionario datasets utilizando el nombre del dataset
    train, test = #<RELLENAR>
    # Ejecutamos el método CNN con el clasificador knn creado anteriormente y los conjuntos obtenidos (train y test)
    # Con _ evitamos recibir los parámetros en variables, para el Test.assertEquals solo comprobaremos la precisión por ahora
    _, accTrain, accTest, _, _ = ejecutaMetodoIS(#<RELLENAR>)
    
Test.assertEquals(round(accTrain, 4), 0.8175, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.8906, "Precisión en test incorrecta para banana")

Como hemos explicado en teoría, CNN está muy influenciado por el orden en el que se cogen las instancias. De la forma en la que lo hemos implementado, dicho orden es aleatorio. Ejecuta el método con el dataset iris tres veces y comprueba la gran diferencia entre los resultados obtenidos.

In [ ]:
# Utilizamos el dataset iris
train, test = datasets['iris']
# Repetimos 3 veces la ejecución, no nos interesa guardar las salidas de ejecutaMetodoIS en este caso
for i in range(0, 3):
    ejecutaMetodoIS(CNN, knn, train, test)

## Reduced Nearest Neighbor (RNN)

RNN es un método que exitiende el anterior (CNN). CNN no asegura que el conjunto de instancias seleccionadas sea mínimo (el menor posible que clasifica correctamente todas los ejemplos de entrenamiento). RNN trata de arreglar ese problema eliminando instancias de la selección realizada por CNN. Para ello, parte de dicha selección y elimina aquellas instancias que no causan que ninguna instancia de $TR$ sea clasificada incorrectamente.

1. $S= S_{CNN}$
2. Para cada ejemplo $e$ en $S$:
    * Comprobamos si eliminar el ejemplo $e$ de $S$ no causa que ninguna instancia de $TR$ sea clasificada incorrectamente
    * En ese caso, eliminamos $e$
    
Las instancias seleccionadas se representan como un vector de booleanos $S$ del tamaño del número de ejemplos ($S[i]= True$ el ejemplo $i$ es seleccionado y $S[i]= False$, el ejemplo no se selecciona.

Para RNN, $S$ comienza siendo el vector devuelto por CNN.

Después, recorremos cada ejemplo en $S$ (con orden aleatorio) y comprobamos si la eliminación de dicho ejemplo provoca que algún ejemplo que no está en $S$ sea fallado por los ejemplos que se quedarían en $S$ (es decir, $S$ menos el ejemplo a eliminar). Si se falla algún ejemplo, la instancia no se elimina, en otro caso se puede desechar. 

In [ ]:
def RNN(X, y, k=1):
    """
    Algoritmo RNN para la selección de instancias. Se parte de la selección obtenida con CNN y se eliminan aquellas instancias
    que no provoquen que se falle ninguna instancia no seleccionada (las seleccionadas se aciertan por definición al estar en el subconjunto)
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Creamos el clasificador knn con el valor de k dado
    knn = #<RELLENAR>

    # En RNN partimos de la selección realizada por CNN y eliminamos ejemplos de la misma
    # S es una máscara de booleanos que indica si el ejemplo en la posición i está seleccionado (True) o no lo está (False)
    # Llamamos a CNN con los parámetros de entrada de RNN
    S = #<RELLENAR>

    # Obtenemos la máscara de instancias no seleccionadas aplicando np.logical_not()
    # No podemos fallar ninguna de estas si queremos eliminar una instancia de S
    notS = #<RELLENAR>
    # Recorremos las instancias seleccionadas para ver si podemos eliminarlas o no
    # Obtenemos para ello los índices de las instancias y las ordenamos aleatoriamente
    indices = np.where(S)[0]
    for i in np.random.permutation(indices):
        # Asumimos que podemos eliminar la instancia y la "eliminamos" temporalmente
        S[i], notS[i] = notS[i], S[i] # Intercambiamos valores, S[i] = False y Snot[i] = True
        # Establecemos el conjunto de train para knn una vez eliminada la instancia (aplicamos la máscara S)
        knn = #<RELLENAR>
        # Obtenemos las salidas con knn.predict para los ejemplos que no están seleccionados (aplicamos la máscara notS)
        salidas = #<RELLENAR>
        # Si todos los ejemplos no seleccionados son acertados (no se falla ninguno) mantenemos la eliminación (no hacemos nada)
        # En otro caso, si fallamos alguna instancia, debemos restablecer la instancia eliminada
        if np.sum(y[notS] != salidas) != 0:
            # Restablecemos la instancia eliminada
            S[i], notS[i] = notS[i], S[i] # Recuperamos valores originales, S[i] = True y Snot[i] = False

    return S

Utiliza la función ejecutaMetodoIS implementada, para comprobar los resultados obtenidos por RNN en cada dataset.

¿Elimina muchos ejemplos? ¿Mejora la precisión en train? ¿y en test? ¿Qué ha pasado con las fronteras de decisión? ¿Hay cambios respecto a CNN?

In [ ]:
# Como usamos aleatorios en RNN establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
np.random.seed(12312)
# Para cada dataset...
for dataset in nombresDatasets:
    # Obtenemos el conjunto de train y el de test del diccionario datasets utilizando el nombre del dataset
    train, test = #<RELLENAR>
    # Ejecutamos el método RNN con el clasificador knn creado anteriormente y los conjuntos obtenidos (train y test)
    # Con _ evitamos recibir los parámetros en variables, para el Test.assertEquals solo comprobaremos la precisión por ahora
    _, accTrain, accTest, _, _ = ejecutaMetodoIS(RNN, knn, train, test)
    
Test.assertEquals(round(accTrain, 4), 0.8118, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.8925, "Precisión en test incorrecta para banana")

## Random Mutation Hill Climbing (RMHC)

RMHC a diferencia de los anteriores es un método de selección híbrido, que trata de buscar el mejor subconjunto de instancias en base a la precisión obtenida por el mismo mediante leave-one-out. El número de instancias seleccionadas en RMHC es fijo (en este caso lo hemos implementado como un porcentaje $s$ del tamaño del conjunto de entrenamiento).

El método es muy simple.

1. $S=$ conjunto aleatorio de instancias de $TR$ de tamaño $s\cdot |TR|$
2. En cada iteración (hasta $iteraciones$)
    * Se intercambia una instancia de $S$ con otra de $TR-S$
    * El cambio se mantiene si la precisión mejora (leave-one-out para obtenerla)
    
En este caso y por facilidad, aunque la salida final sigue siendo la máscara que indica las instancias seleccionadas, para implementar el algoritmo usamos dos listas, una de instancias seleccionadas  y otra de no seleccionadas. De esta forma resulta más sencillo intercambiar instancias de una a otra (ambas mantendrán siempre el mismo tamaño). Cada lista (o array) contiene los índices de las instancias seleccionadas o no seleccionadas.

Inicialmente dichas listas son aleatorias y en cada iteración se elige un elemento de cada lista al azar para ser intercambiados. Se realiza el cambio, se comprueba la precisión mediante leave-one-out, y en caso de mejora el cambio se mantiene. En otro caso, el cambio se deshace.

In [ ]:
def RMHC(X, y, s=0.1, iteraciones=1000, k=1):
    """
     Algoritmo RMHC (Random Mutation Hill Climbing) para la selección de instancias.
      Se comienza con una selección aleatoria de s * nEjemplos instancias. Para cada iteración, se elige una instancia
      seleccionada y una no seleccionada para ser intercambiadas. Si el intercambio mejora la precisión (leave-one-out) sobre train
      se mantiene el cambio, sino se deshace
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param s: Porcentaje de instancias a ser seleccionadas (s es un valor entre 0 y 1)
    :param iteraciones: Número de iteraciones (intercambios) a probar
    :param k: Valor de k a utilizar en ENN
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Creamos el clasificador knn con el valor de k dado
    knn = #<RELLENAR>

    # En este caso vamos a mantener dos vectores de enteros con los índices de las instancias seleccionadas y no seleccionadas
    # Establecemos el número de instancias a seleccionar (es fijo)
    nSel = int(s * X.shape[0])
    # Obtenemos una permutación aleatoria de todos los índices disponibles 
    # (un array de 0 a X.shape[0] con valores ordenados aleatoriamente)
    permute = np.random.permutation(X.shape[0])
    # Cogemos como seleccionadas las instancias correspondientes a los primeros nSel índices  y no seleccionadas el resto
    seleccionadas = permute[:nSel]
    noSeleccionadas = permute[nSel:]

    # Calculamos la preicisión con la selección inicial
    # Establecemos el conjunto de entrenamiento de kNN con fit y las instancias seleccionadas
    knn = knn.fit(X[seleccionadas, :], y[seleccionadas])
    # Obtenemos las salidas de todo el conjunto con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
    salidas = #<RELLENAR>
    # Calculamos la precisión con metrics.accuracy_score
    acc = #<RELLENAR>

    # Comenzamos las iteraciones:
    #   En cada una, intercambiamos una instancia seleccionada con una que no lo está y comprobamos si mejora la precisión
    for i in range(0, iteraciones):
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a eliminar de las seleccionadas
        quitar = np.random.randint(0, nSel)
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a añadir a las seleccionadas
        poner = np.random.randint(0, noSeleccionadas.size)
        # Guardamos el ejemplo eliminado
        aux = seleccionadas[quitar]
        # Añadimos la nueva instancia seleccionada sustituyendo la que se elimina
        seleccionadas[quitar] = noSeleccionadas[poner]
        # Establecemos el nuevo conjunto de instancias seleccionadas para kNN
        knn = #<RELLENAR>

        # Calculamos la precisión de la solución actual
        # Obtenemos las salidas de todo el conjunto con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
        salidas = #<RELLENAR>
        # Calculamos la precisión con metrics.accuracy_score
        accNew = #<RELLENAR>

        # Si la precisión actual no mejora a la anterior, devolvemos la instancia eliminada a las seleccionadas
        # Sino si la precisión de la solución actual es mejor que la anterior, guardamos la precisión y
        #  añadimos la instancia eliminada a las no seleccionadas
        if accNew < acc:
            seleccionadas[quitar] = aux
        else:
            acc = accNew
            noSeleccionadas[poner] = aux

        if i % 100 == 0:
            print("precision en iteracion {}: {}".format(i, acc))

    # A partir de las seleccionadas creamos la máscara de instancias seleccionadas 
    # donde True en la posición i indica que la instancia i es seleccionada
    S = np.zeros( (X.shape[0]), dtype=bool)
    S[seleccionadas] = True

    return S

Utiliza la función ejecutaMetodoIS implementada, para comprobar los resultados obtenidos por RMH en cada dataset.

¿Elimina muchos ejemplos? ¿Mejora la precisión en train? ¿y en test? ¿Qué ha pasado con las fronteras de decisión? ¿Notas cambios respecto a los métodos de edición o condensación?

In [ ]:
# Como usamos aleatorios en RMHC establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
np.random.seed(12312)
# Para cada dataset...
for dataset in nombresDatasets:
    # Obtenemos el conjunto de train y el de test del diccionario datasets utilizando el nombre del dataset
    train, test = #<RELLENAR>
    # Ejecutamos el método RMHC con el clasificador knn creado anteriormente y los conjuntos obtenidos (train y test)
    # Con _ evitamos recibir los parámetros en variables, para el Test.assertEquals solo comprobaremos la precisión por ahora
    _, accTrain, accTest, _, _ = ejecutaMetodoIS(#<RELLENAR>)
    
Test.assertEquals(round(accTrain, 4), 0.8118, "Precisión en train incorrecta para banana")
Test.assertEquals(round(accTest, 4), 0.8925, "Precisión en test incorrecta para banana")

# Comparación de los métodos con kNN
Ya tenemos 5 métodos de selección de instancias implementados. Ahora vamos a realizar una pequeña comparativa de todos ellos a la vez.

En primer lugar y para poder continuar utilizando la función ejecutaMetodoIS, vamos a implementar una función muy tonta que simplemente no realice la selección (devuelve un array de booleanos con todo a True). Esta función nos servirá para representar el resultado del modelo sin selección posteriormente.

In [ ]:
def sinSeleccion(X, y):
    # Crea la máscara S como un array de booleanos del tamaño del conjunto de entrenamiento con todos los valores a True
    S = #<RELLENAR>
    return S

Por rapidez en la ejecución, vamos a realizar la comparativa con el dataset iris (pero luego puedes cambiar a banana para ver los resultados).

Establecemos una lista con los métodos de selección que vamos a testear. Posteriormente, creamos una serie de matrices y vectores vacías para obtener los resultados de cada método de selección de instancias (instancias seleccionadas, precisión en train, precisión en test y reducción).

Con estas listas luego seremos capaces de montar una tabla con la que comparar rápidamente los resultados.

Al ejecutar el siguiente código podrás observar los resultados de cada método gráficamente.

In [ ]:
# Obtenemos los conjuntos de train y test de iris
train, test = datasets[#<RELLENAR>]
# Establecemos una lista con las funciones de los métodos de selección
metodos = (sinSeleccion, ENN, AllKNN, CNN, RNN, RMHC)
# Obtenemos el número de métodos con len
nMetodos = #<RELLENAR>

# Inicializamos las variables para guardar los resultados
S = np.zeros((nMetodos, train.data.shape[0]))
accTrain = np.zeros((nMetodos))
accTest = np.zeros((nMetodos))
reduction = np.zeros((nMetodos))
for i, metodo in enumerate(metodos):
    # Ejecutamos cada método con knn y los conjuntos de train y test de iris, ponemos verbose=False
    S[i, :], accTrain[i], accTest[i], reduction[i], _ = ejecutaMetodoIS(metodo, knn, train, test, verbose=False)

Ahora que tenemos los resultados en vectores de numpy, los vamos a utilizar para crear un dataframe de Panda donde podamos ver para cada método la precisión obtenida en train y test además de la capacidad de reducción del mismo.

¿Con qué método te quedarías? ¿Gana alguna selección de instancias a no realizarla?

In [ ]:
# Concatenamos horizontalmente los datos de entrada con los de salida (hstack de las variable accTrain, accTest y reduction
# con su correspondiente reshape a -1, 1)
datosPD = np.hstack((#<RELLENAR>))
# Creamos los índices de las filas que son los nombres de los métodos. Hacemos uso de la función map para obtener los nombres
filas = map(lambda metodo: metodo.__name__, metodos)
# Creamos los nombres de las columnas
nombreColumnas = ['Acc Train', 'Acc Test', 'Reducción']

# Importamos la librería de pandas ya que no lo  hemos hecho anteriormente
import pandas as pd
# Generamos el dataframe con data=datosPD, index=filas y columns=nombreColumnas
datosPanda = pd.DataFrame(data=datosPD, index=filas, columns=nombreColumnas)
# Mostramos el dataframe generado
print(datosPanda)

# Comparación de los métodos con Árboles de Decisión

Hasta ahora hemos utilizado todos los métodos de selección de instancias con kNN (sería el caso en el que a los métodos los podemos denominar como métodos de selección de prototipos).

Sin embargo, algunos de ellos son también útiles para lo que denominamos como Training Set Selection, en el que la selección se realiza de manera general para cualquier método de aprendizaje. En este caso, nosotros vamos a probarlos con los árboles de decisión.[Link a los árboles de decisión en scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

Se trata de observar si su comportamiento es bueno con los árboles, si todos ellos funcionan o solo algunos y cuáles en ese caso. Veremos además como una de las propiedades interesante de la selección de instancias en este tipo de modelos es que los simplifican, reduciendo en gran medida el número de reglas obtenidas.

In [ ]:
# Crea un árbol de decisión de scikit con el criterio "entropy"
dt = #<RELLENAR>

# Obtenemos los conjuntos de train y test de iris primero y probar con banana despúes
train, test = datasets[#<RELLENAR>]

# Establecemos una lista con las funciones de los métodos de selección
metodos = (sinSeleccion, ENN, AllKNN, CNN, RNN, RMHC)
# Obtenemos el número de métodos con len
nMetodos = #<RELLENAR>

# Inicializamos las variables para guardar los resultados
S = np.zeros((nMetodos, train.data.shape[0]))
accTrain = np.zeros((nMetodos))
accTest = np.zeros((nMetodos))
reduction = np.zeros((nMetodos))
nRules = np.zeros((nMetodos))
for i, metodo in enumerate(metodos):
    S[i, :], accTrain[i], accTest[i], reduction[i], nRules[i] = ejecutaMetodoIS(metodo, dt, train, test, verbose=False)

# Concatenamos horizontalmente los datos de entrada con los de salida (hstack de las variable accTrain, accTest, reduction y nRules
# con su correspondiente reshape a -1, 1)
datosPD = np.hstack((#<RELLENAR>))
# Creamos los índices de las filas que son los nombres de los métodos. Hacemos uso de la función map para obtener los nombres
filas = map(lambda metodo: metodo.__name__, metodos)
# Creamos los nombres de las columnas
nombreColumnas = ['Acc Train', 'Acc Test', 'Reducción', 'Num. Reglas']

# Generamos el dataframe con data=datosPD, index=filas y columns=nombreColumnas
datosPanda = pd.DataFrame(data=datosPD, index=filas, columns=nombreColumnas)
print(datosPanda)

En el caso de iris podemos ver como con muchas menos reglas (menos de la mitad) somos capaces de obtener la misma precisión en test y la mayoría de los métodos funcionan bien (veremos que no siempre ocurre lo mismo en el siguiente estudio).

En el caso de banana podemos ver como los métodos de condensación (CNN y RNN) no funcionan muy bien. Sin embargo, los métodos de edición reducen mucho el número de reglas y llegan a mejorar la precisión en test. En el caso de RMHC, el número de reglas es más de 10 veces menor y según la ejecución (depende de la aleatoriedad) es capaz de mejorar la precisión (pero tiene la pega de que es poco robusto).

# Selección de Instancias con KEEL: Comparativa

Para terminar con la práctica vamos a estudiar algún método más de selección de instancias. Pero en este caso haremos uso de los disponibles en la herramienta KEEL ([Link a KEEL](http://keel.es/)). KEEL es una herramienta con una gran cantidad de algoritmos de minería de datos y en particular, de preprocesamiento.

A continuación hemos generado un wrapper que es capaz de llamar a los métodos implementados en KEEL y utilizarlos desde Python. En este caso no obtendremos una máscara de ejemplos seleccionados, sino el propio dataset con las instancias seleccionadas. Es por ello que en esta función necesitamos el min_max_scaler creado al principio de la práctica para cada dataset, ya que KEEL internamente normaliza el conjunto de datos pero a la hora de escribirlo, escribe a fichero las instancias originales.

Gran parte de este wrapper es similar a ejecutaMetodoIS y por ello, los pasos para obtener la precisión mediante leave-one-out, la precisión en test mediante precit, el ratio de reducción y mostrar la gráfica son iguales.

In [ ]:
def ejecutaMetodoIS_KEEL(dataset, nombreMetodo, clasificador, train, test, min_max_scaler, verbose=True):
    """
    Esta función se encajar de ejecutar un método de selección de instancias y dar los resultados sobre train y test
    con el clasificador introducido como parámetro
    :param funcionMetodo: Función con el método de selección de instancias a ejecutar
    :param clasificador: Instancia del clasificador a utilizar para obtener la precisión en train y test
    :param train: Conjunto de datos de entrenamiento leído del formato keel con campos data y target
    :param test: Conjunto de datos de test leído del formato keel con campos data y target
    :return: tupla con la máscara de ejemplos seleccionados, la precisión en train, precisión en test y porcentaje de reducción obtenido
    """
    if verbose:
        print("Ejecutando " + nombreMetodo + "...")
    # Preparamos la ejecución para KEEL
    ejecutable = 'KEEL/IS-' + nombreMetodo + '.jar'
    # Los parámetros de configuración se encuentran en los ficheros nombreMetodo+'Conf.txt'
    fileConfig = 'KEEL/' + nombreMetodo + 'Conf.txt'
    # Para ejecutar el método en KEEL debemos copiar el fichero de entrenamiento a uno con nombre 'training2.txt'
    # A su vez, el fichero de salida procesado con las instancias seleccionadas estará en 'training.txt'
    fileTr = 'datasets/' + dataset + '/' + dataset + '-5dobscv-2tra.dat'
    copyfile(fileTr, 'training2.txt')
    fileDataPreproTrain = 'training2.txt'
    # Creamos el comando y lo ejecutamos, la salida va al fichero 'error.txt'
    command1 = ["java", "-jar", ejecutable, fileConfig]
    with open("error.txt", "w") as fE:
        subprocess.check_call(command1, stderr=fE, stdout=fE)
    # Las instancias seleccionadas están en 'training.txt', las leemos con ds.lecturaDatos
    trainPrepro, originalPreproTr = ds.lecturaDatos('training.txt')
    if verbose:
        print("Número de ejemplos por clase en las instancias seleccionadas:")
        print(np.bincount(trainPrepro.target))

    # Debemos normalizar los datos de las instancias seleccionadas. En KEEL esta normalización
    # se realiza al hacer la selección, pero el dataset guardado a fichero tiene los rangos originales
    # Debemos normalizar con min_max_scaler (recibido por parámetro) los datos en trainPrepro.data
    trainPrepro.data = #<RELLENAR>
        
    # Entrenamos el clasificador correspondiente solo con los ejemplos seleccionados en trainPrepro
    clasificador = clasificador.fit(trainPrepro.data, trainPrepro.target)
    # Obtenemos las salidas para train mediante leaveOneOut (aplicado solo para kNN, sino el árbol se aprende en train directamente)
    predictionTrain = leaveOneOut(clasificador, train.data, train.target)
    # Obtenemos la precisión en train con las salidas obtenidas
    accTrain = metrics.accuracy_score(predictionTrain, train.target)
    # Obtenemos las salidas para test utilizando la función predict del clasificador
    predictionTest = clasificador.predict(test.data)
    # Obtenemos la precisión en test utilizando las métricas de sklearn
    accTest = metrics.accuracy_score(predictionTest, test.target)
    # Calculamos el porcentaje de reducción
    reduction = 100.0 - trainPrepro.target.size / float(train.target.size)*100.0
        
    
    if verbose:
        print("Dataset " + dataset)
        print("Resultados " + nombreMetodo + " python")
        print("Precisión en train: {}".format(accTrain))
        print("Precisión en test: {}".format(accTest))
        print("Reducción " + nombreMetodo + ": {} de {}".format(S.sum(), S.size))
        print("Reducción: %2.2f%%" % reduction)
        # En caso de ser un árbol, imprimimos el número de reglas obtenidas
        if type(clasificador) == tree.DecisionTreeClassifier:
            print("Número de reglas: {}".format(clasificador.tree_.node_count))

    # Mostramos los resultados obtenidos por el método gráficamente
    mostrar(clasificador, train.data, train.target, test.data, test.target,  nombreMetodo, trainPrepro.data, trainPrepro.target)

    # Si es un árbol, devolvemos el número de reglas obtenidas
    nReglas = -1
    if type(clasificador) == tree.DecisionTreeClassifier:
        nReglas = clasificador.tree_.node_count
    return accTrain, accTest, reduction, nReglas

Una vez creado el wrapper que permite llamar a KEEL (requiere tener los fichero .jar en la carpeta KEEL junto al notebook), ya podemos comprobar los resultados que obtenemos.

Los métodos que vamos a probar con KEEL son (KEEL tiene muchos más métodos pero no todos pueden usarse mediante el wrapper):

* **AllKNN**: Elimina ejemplos mal clasificados con algún valor de k entre 1 y K
* **ENN**: Elimina ejemplos mal classificados con 3NN
* **CHC**: Es un algoritmo evolutivo que busca la mejor selección de instancias en base al ratio de reducción y la clasificación con leave-one-out

Vamos a realizar los experimentos con kNN y el árbol de decisión. Trabajaremos con ambos datasets y para cada uno obtendremo la misma tabla que obteníamos anteriormente. Atención especial merece el comportamiento del algoritmo CHC, que con una gran capacidad de reducción (no es fija como en RMHC) obtiene muchas veces los mejores resultados (¡OJO! ¿También con árboles de decisión? ¿Qué ocurre en este caso?...)

In [ ]:
metodosKEEL = ('AllKNN', 'ENN', 'CHC')
    
# Crea un clasificador knn con k = 1
knn = neighbors.KNeighborsClassifier(n_neighbors=1)
# Crea un árbol de decisión de scikit con el criterio "entropy"
dt = #<RELLENAR>

# Obtenemos el número de métodos a ejecutar con len
nMetodos = #<RELLENAR>
for clasificador in [knn, dt]:
    print('Trabajando con clasificador ' + str(clasificador.__class__))
    for dataset in nombresDatasets:
        print('Trabajando con dataset ' + dataset)
        train, test = datasets[dataset] # Probar con banana y con iris
        accTrain = np.zeros((nMetodos))
        accTest = np.zeros((nMetodos))
        reduction = np.zeros((nMetodos))
        nRules = np.zeros((nMetodos))

        for i, metodo in enumerate(metodosKEEL):
            accTrain[i], accTest[i], reduction[i], nRules[i] = ejecutaMetodoIS_KEEL(dataset, metodo, clasificador, train, test, min_max_scaler[dataset], verbose=False)

        # Concatenamos horizontalmente los datos de entrada con los de salida (hstack de las variable accTrain, accTest, reduction y nRules
        # con su correspondiente reshape a -1, 1)
        datosPD = np.hstack((#<RELLENAR>))
        # Creamos los nombres de las columnas
        nombreColumnas = ['Acc Train', 'Acc Test', 'Reducción', 'Num. Reglas']

        # Generamos el dataframe con data=datosPD, index=filas y columns=nombreColumnas
        datosPanda = pd.DataFrame(data=datosPD, index=metodosKEEL, columns=nombreColumnas)
        print(datosPanda)
        

Normalmente CHC no suele funcionar bien con los árboles de decisión porque reduce tanto el número de instancias que el árbol no es capaz de aprender. Para kNN (con $k=1$ esto no tiene porque ser una restricción). Además, hay que tener en cuenta que a la hora de la reducción CHC calcula la precisión con kNN y por tanto no tiene porque ser extensible su comportamiento a otros clasificadores.